In [13]:
import sqlalchemy as db
import pandas as pd
import os
import csv
import seaborn as sns
import matplotlib.pyplot as plt

## Connecting to database

In [14]:
# connecting to database sql_challenge created in postgresql
engine = db.create_engine('postgresql://localhost:5432/project2')
connection = engine.connect()
metadata = db.MetaData()

In [15]:
#to find all the tables for the database (it should be empty)
engine.table_names()

/var/folders/ld/_spx9cwj74z4zm921xp1zssh0000gn/T/ipykernel_2268/3558990862.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


[]

# Creating Table


In [16]:
#creating new empty tables 
election = db.Table('election', metadata,
              db.Column('County', db.String(20), primary_key=True, nullable=False), 
              db.Column('State', db.String(20),primary_key=True, nullable=False),
                db.Column('Republic', db.Integer()), 
                db.Column('Democrate', db.Integer()), 
                db.Column('Win', db.String(20)) 
              )

census = db.Table('census', metadata,
              db.Column('County', db.String(), primary_key=True, nullable=False), 
              db.Column('State', db.String(20)),
                  db.Column('Population', db.Float()), 
                  db.Column('Median Age', db.Float()),
                  db.Column('Household Income', db.Float()), 
                  db.Column('Poverty Rate', db.Float()), 
                  db.Column('Unemployment Rate', db.Float()), 
                  db.Column( 'High School Rate', db.Float()), 
                  db.Column('College Rate', db.Float()), 
                  db.Column('Uneducated Rate', db.Float()), 
                  db.Column('White Population Rate', db.Float()), 
                  db.Column('Black Population Rate', db.Float()),
                  db.Column('Hispanic Population Rate', db.Float()),
                  db.Column('Asian Population Rate', db.Float())                  
              )


vaccine = db.Table('vaccine', metadata,
                   db.Column('Date', db.Date()),
                  db.Column('County', db.String(20), primary_key=True, nullable=False), 
                  db.Column('State', db.String(10),primary_key=True, nullable=False),  
                   db.Column('Series_Complete_Pop_Pct', db.Float()),
                   db.Column('Fully_Vaccinated', db.Integer()),
                   db.Column('Fully_Vaccinated_12+', db.Float()),
                   db.Column('Fully_Vaccinated_18+', db.Float()),
                   db.Column('Fully_Vaccinated_65+', db.Float()),
                   db.Column('Administered_Dose1_Recip', db.Integer()),
                   db.Column('Partially_Vaccinated_12+', db.Float()),
                   db.Column('Partially_Vaccinated_18+', db.Float()),
                   db.Column('Partially_Vaccinated_65+', db.Float()),
        
              )

metadata.create_all(engine) #Creates the tables

In [17]:
#to find all the tables for the database(it will show all the tables created)
engine.table_names()

/var/folders/ld/_spx9cwj74z4zm921xp1zssh0000gn/T/ipykernel_2268/1032459306.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['election', 'census', 'vaccine']

In [18]:
#getting the tables from database
election = db.Table('election', metadata, autoload=True, autoload_with=engine)
census = db.Table('census', metadata, autoload=True, autoload_with=engine)
vaccine = db.Table('vaccine', metadata, autoload=True, autoload_with=engine)


## Inserting values in tables from csv files 


In [ ]:
#For table election
path_election=os.path.join('cleaned_data', 'election_data.csv')

with open(path_election, 'r', encoding="utf-8") as tit:
    next(tit) #removing header
    csv_tit = csv.reader(tit, delimiter=',')
    engine.execute(
        titles.insert(),
        [{"title_id": row[0], "title": row[1]} 
            for row in csv_tit]
    )


#For table census
path_census=os.path.join('data', 'departments.csv')

with open(path_census, 'r', encoding="utf-8") as dept:
    next(dept) #removing header
    csv_dept = csv.reader(dept, delimiter=',')
    engine.execute(
        departments.insert(),
        [{"dept_no": row[0], "dept_name": row[1]} 
            for row in csv_dept]
    )

#For table cdc_vaccine
path_vaccine=os.path.join('data', 'employees.csv')

with open(path_cdc_vaccine, 'r', encoding="utf-8") as emp:
    next(emp) #removing header
    csv_emp = csv.reader(emp, delimiter=',')
    engine.execute(
        employees.insert(),
        [{"emp_no": row[0], "emp_title_id": row[1],
         "birth_date": row[2], "first_name": row[3],
         "last_name": row[4], "sex": row[5],
         "hire_date": row[6]} for row in csv_emp]
    )


In [ ]:
# Print full table metadata for employees table
print(repr(metadata.tables['employees']))

In [ ]:
#can also use c instead of columns
employees.c.keys()

# Querying


### 1. To list employee number, last name, first name, sex, and salary

In [ ]:
### sql
sql_1= '''SELECT e.emp_no, e.last_name, e.first_name, e.sex, s.salary 
       FROM employees AS e
       JOIN salaries AS s
       ON e.emp_no=s.emp_no'''

In [ ]:
#using text
results_1=connection.execute(db.text(sql_1)).fetchall()
df_1 = pd.DataFrame(results_1, columns=(['Emp_No', 'Last_Name', 
                                         "First_Name",'Sex','Salary']))
df_1


In [ ]:
#using sql_alchemy
res_1=connection.execute(db.select([employees.c.emp_no,
                                   employees.c.last_name,
                                   employees.c.first_name,
                                   employees.c.sex,
                                   salaries.c.salary])
                         .join(salaries, employees.c.emp_no==salaries.c.emp_no)
                         ).fetchall()
df_alc1 = pd.DataFrame(res_1, columns=(['Emp_No', 'Last_Name', 
                                         "First_Name",'Sex','Salary']))
df_alc1

### 2. To list first name, last name, and hire date for employees who were hired in 1986.

In [ ]:
sql_2='''SELECT first_name, last_name, hire_date
        FROM employees
        WHERE hire_date BETWEEN '1986-01-01' AND '1986-12-31' '''


In [ ]:
#using text
results_2=connection.execute(db.text(sql_2)).fetchall()
df_2 = pd.DataFrame(results_2, columns=(['first_name', 'last_name', 'hire_date']))
df_2

In [ ]:
#using sqlalchemy
res_2=connection.execute(db.select([employees.c.first_name,
                                   employees.c.last_name,
                                   employees.c.hire_date])
                        .where(employees.c.hire_date.between('1986-01-01','1986-12-31'))
                        ).fetchall()
df_alc2 = pd.DataFrame(res_2, columns=(['first_name', 'last_name', 'hire_date']))
df_alc2

### 3. To list the manager of each department with the following information: department number, department name, the manager's employee number, last name, first name.

In [ ]:
sql_3='''SELECT  d.dept_no, d.dept_name, e.emp_no, e.last_name, e.first_name 
        FROM departments AS d
        JOIN dept_manager AS dm
        ON d.dept_no=dm.dept_no
        JOIN employees AS e
        ON dm.emp_no=e.emp_no'''

In [ ]:
#using text
results_3=connection.execute(db.text(sql_3)).fetchall()
df_3 = pd.DataFrame(results_3, columns=(['dept_no', 'dept_name', 
                                         'emp_no', 'last_name', 'first_name']))
df_3

In [ ]:
#using sql_alchemy
res_3=connection.execute(db.select([departments.c.dept_no,
                                    departments.c.dept_name,
                                    employees.c.emp_no,
                                   employees.c.last_name,
                                   employees.c.first_name])
                         .join(dept_manager, departments.c.dept_no==dept_manager.c.dept_no)
                         .join(employees, dept_manager.c.emp_no==employees.c.emp_no)
                         ).fetchall()
df_alc3 = pd.DataFrame(res_3, columns=(['dept_no', 'dept_name', 
                                         'emp_no', 'last_name', 'first_name']))
df_alc3

### 4. To list the department of each employee with the following information: employee number, last name, first name, and department name.

In [ ]:
sql_4='''SELECT e.emp_no, e.last_name, e.first_name, d.dept_name
        FROM employees AS e
        JOIN dept_emp AS de
        ON e.emp_no=de.emp_no
        JOIN departments AS d
        on de.dept_no=d.dept_no'''

In [ ]:
#using text
results_4=connection.execute(db.text(sql_4)).fetchall()
df_4 = pd.DataFrame(results_4, columns=(['emp_no', 'last_name', 
                                         'first_name', 'dept_name']))
df_4

In [ ]:
#using sql_alchemy
res_4=connection.execute(db.select([employees.c.emp_no,
                                   employees.c.last_name,
                                   employees.c.first_name,
                                   departments.c.dept_name])
                         .join(dept_emp, employees.c.emp_no==dept_emp.c.emp_no)
                         .join(departments, dept_emp.c.dept_no==departments.c.dept_no)
                         ).fetchall()
df_alc4 = pd.DataFrame(res_4, columns=(['emp_no', 'last_name', 
                                         'first_name', 'dept_name']))
df_alc4

### 5. To list first name, last name, and sex for employees whose first name is "Hercules" and last names begin with "B."

In [ ]:
sql_5='''SELECT first_name, last_name, sex
        FROM employees
        WHERE first_name='Hercules' AND last_name LIKE'B%' '''

In [ ]:
#using text
results_5=connection.execute(db.text(sql_5)).fetchall()
df_5 = pd.DataFrame(results_5, columns=(['first_name', 'last_name', 'sex']))
df_5

In [ ]:
#using sqlalchemy 
res_5=connection.execute(db.select([employees.c.first_name,
                                    employees.c.last_name, employees.c.sex])
                        .where(db.and_(employees.c.first_name=='Hercules', 
                                           employees.c.last_name.like('B%')))
                        ).fetchall()
df_alc5 = pd.DataFrame(res_5, columns=(['first_name', 'last_name', 'sex']))
df_alc5

### 6. To list all employees in the Sales department, including their employee number, last name, first name, and department name.

In [ ]:
sql_6='''SELECT e.emp_no, e.last_name, e.first_name, e.sex, d.dept_name
        FROM employees AS e
        JOIN dept_emp AS de
        ON e.emp_no=de.emp_no
        JOIN departments AS d
        ON de.dept_no=d.dept_no
        WHERE d.dept_name='Sales' '''

In [ ]:
#using text
results_6=connection.execute(db.text(sql_6)).fetchall()
df_6 = pd.DataFrame(results_6, columns=(['emp_no', 'last_name', 
                                         'first_name','sex','dept_name']))
df_6

In [ ]:
#using sql_alchemy
res_6=connection.execute(db.select([employees.c.emp_no,
                                   employees.c.last_name,
                                   employees.c.first_name,
                                   employees.c.sex,
                                   departments.c.dept_name])
                         .join(dept_emp, employees.c.emp_no==dept_emp.c.emp_no)
                         .join(departments, dept_emp.c.dept_no==departments.c.dept_no)
                         .where(departments.c.dept_name=='Sales')
                         ).fetchall()
df_alc6 = pd.DataFrame(res_6, columns=(['emp_no', 'last_name', 
                                         'first_name','sex','dept_name']))
df_alc6

### 7. To list all employees in the Sales and Development departments, including their employee number, last name, first name, and department name.

In [ ]:
sql_7='''SELECT e.emp_no, e.last_name, e.first_name, e.sex, d.dept_name
        FROM employees AS e
        JOIN dept_emp AS de
        ON e.emp_no=de.emp_no
        JOIN departments AS d
        ON de.dept_no=d.dept_no
        WHERE d.dept_name='Sales' OR d.dept_name='Development' '''

In [ ]:
#using text
results_7=connection.execute(db.text(sql_7)).fetchall()
df_7 = pd.DataFrame(results_7, columns=(['emp_no', 'last_name', 
                                         'first_name','sex','dept_name']))
df_7

In [ ]:
#using sql_alchemy
res_7=connection.execute(db.select([employees.c.emp_no,
                                   employees.c.last_name,
                                   employees.c.first_name,
                                   employees.c.sex,
                                   departments.c.dept_name])
                         .join(dept_emp, employees.c.emp_no==dept_emp.c.emp_no)
                         .join(departments, dept_emp.c.dept_no==departments.c.dept_no)
                         .where(db.or_(departments.c.dept_name=='Sales', 
                                      departments.c.dept_name=='Development'))
                         ).fetchall()
df_alc7 = pd.DataFrame(res_7, columns=(['emp_no', 'last_name', 
                                         'first_name','sex','dept_name']))
df_alc7

### 8. To list the frequency count of employee last names, i.e., how many employees share each last name in descending order

In [ ]:
sql_8='''SELECT last_name, COUNT(last_name) AS count
        FROM employees
        GROUP BY last_name
        ORDER BY count DESC'''

In [ ]:
#using text
results_8=connection.execute(db.text(sql_8)).fetchall()
df_8 = pd.DataFrame(results_8, columns=(['last_name','count']))
df_8

In [ ]:
#using sqlalchemy 
res_8=connection.execute(db.select([employees.c.last_name,
                                    db.func.count(employees.c.last_name)])
                        .group_by(employees.c.last_name)
                        .order_by(db.desc(db.func.count(employees.c.last_name)))
                        ).fetchall()
df_alc8 = pd.DataFrame(res_8, columns=(['last_name','count']))
df_alc8

# Plotting

In [ ]:
#histogram to visualize the most common salary ranges for employees.
fig, ax=plt.subplots(figsize=(12,12))
df_1['Salary'].plot(kind='hist', color='r',ax=ax)
ax.set(xlabel='Salary($)', title='Salary range for employees')
plt.show()

In [ ]:
#histogram to visualize the most common salary ranges for employees
fig, ax=plt.subplots(figsize=(12,12))
sns.histplot(x='Salary', data=df_1,kde=True,hue="Sex", ax=ax)
ax.set(xlabel='Salary($)', title='Salary range for employees')
plt.show()

### Most common lastname

In [ ]:
most_common_lastname=df_8.nlargest(20, 'count')
fig, ax=plt.subplots(figsize=(15,15))
sns.barplot(x='last_name', y='count',
            data=most_common_lastname, alpha=0.9,ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.show()

### Bar chart of average salary by title.

In [ ]:
#Create a bar chart of average salary by title.
sql_9='''SELECT e.emp_no, e.last_name, e.first_name, t.title, s.salary
        FROM employees AS e
        JOIN salaries AS s
        ON e.emp_no=s.emp_no
        JOIN titles AS t
        ON e.emp_title_id=t.title_id'''
results_9=connection.execute(db.text(sql_9)).fetchall()
df_9 = pd.DataFrame(results_9, columns=(['emp_no','last_name',
                                         'first_name', 'title', 'salary']))
df_9

In [ ]:
fig, ax=plt.subplots(figsize=(15,15))
sns.barplot(x='title', y='salary',
            data=df_9, alpha=0.9,ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
plt.show()

## Bar and pie plots for Departments

In [ ]:
dep=pd.DataFrame(df_4.groupby('dept_name')['emp_no'].count())
dep.columns=['Total employees']

In [ ]:
fig,ax=plt.subplots(figsize=(12,12))
dep.plot(kind='bar', color='r',ax=ax)
ax.set(xlabel='Departments', ylabel='Count')
ax.legend()
plt.show()

In [ ]:
plt.style.use('ggplot')
fig,ax=plt.subplots(figsize=(12,12))
dep.plot(kind='pie', y='Total employees',autopct='%1.0f%%', ax=ax)
plt.show()